In [11]:
import wx
import numpy as np
import pandas as pd
import glob
import subprocess as sp
import os
from pathlib import Path
from matplotlib import pyplot as plt
from ellipse import LsqEllipse
import cv2
import math
import scipy.stats as stats
from tqdm import tqdm
from matplotlib.patches import Ellipse
from bokeh.plotting import figure, show
from bokeh.models import CustomJS, Slider
import bokeh.layouts

In [3]:
def oe_events_parser(open_ephys_csv_path, channel_names, export_path=None):
    """
    :param open_ephys_csv_path: The path to an open ephys analysis tools exported csv (using TrialReporter.ipynb)
    :param channel_names: a dictionary of the form -
                    { 1 : 'channel name' (L_eye_camera)
                      2 : 'channel name' (Arena_TTL)
                      etc..
                    }
    :param export_path: default None, if a path is specified a csv file will be saved
    :returns open_ephys_events: a pandas DataFrame object where each column has the ON events of one channel
                                and has a title from channel_names
    :returns open_ephys_off_events: same but for the OFF states (only important for the logical start-stop signal)
    """

    # Infer the active channels:
    df = pd.read_csv(open_ephys_csv_path)
    channels = df['channel'].to_numpy(copy=True)
    channels = np.unique(channels)
    df_onstate = df[df['state']==1] #cut the df to represent only rising edges
    df_offstate = df[df['state']==0] # This one is important for the ON/OFF signal of the arena
    list = []
    off_list= []
    for chan in channels: #extract a pandas series of the ON stats timestamps for each channel
        Sname = channel_names[chan]
        s = pd.Series(df_onstate['timestamp'][df_onstate['channel'] == chan], name=Sname)
        offs = pd.Series(df_offstate['timestamp'][df_offstate['channel'] == chan], name=Sname)
        list.append(s)
        off_list.append(offs)
    open_ephys_events = pd.concat(list, axis=1)
    open_ephys_off_events = pd.concat(off_list, axis=1)
    if export_path is not None :
        if not export_path in os.listdir(open_ephys_csv_path.split('events.csv')[0][:-1]):
            open_ephys_events.to_csv(export_path)
    return open_ephys_events , open_ephys_off_events

def convert_h264_mp4(path):
    files_to_convert = glob.glob(path + r'\**\*.h264', recursive=True)
    converted_files = glob.glob(path + r'\**\*.mp4', recursive=True)
    for file in files_to_convert:
        fps = file[file.find('hz') - 2:file.find('hz')]
        if len(fps) != 2:
            fps = 60
            print('could not determine fps, using 60...')
        if not str(fr'{file[:-5]}.mp4') in converted_files:
            sp.run(f'MP4Box -fps {fps} -add {file} {file[:-5]}.mp4')
            print(fr'{file} converted ')
        else:
            print(f'The file {file[:-5]}.mp4 already exists, no conversion necessary')

def validate_no_framedrop(path):
    videos_to_inspect = glob.glob(path + r'\**\*.mp4', recursive=True)
    timestamps_to_inspect = glob.glob(path + r'\**\*.csv', recursive=True)
    for vid in range(len(videos_to_inspect)):
        timestamps = pd.read_csv(timestamps_to_inspect[vid])
        num_reported = timestamps.shape[0]
        cap = cv2.VideoCapture(videos_to_inspect[vid])
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print(f'The video named {os.path.split(videos_to_inspect[vid])[1]} has reported {num_reported} frames '
              f'and has {length} frames, it has dropped {num_reported - length} frames')
        cap.release()

def stamp_diff_videos(path_to_stamp,stamp):
    videos_to_stamp = glob.glob(path_to_stamp + r'\**\*.mp4', recursive=True)
    for vid in videos_to_stamp:
        os.rename(vid, fr'{vid[:-4]}_{stamp}{vid[-4:]}')

def get_frame_timeseries(df,channel):
    index_range = range(0,len(df[channel][df[channel].notna()]))
    timeseries = pd.Series(df[channel][df[channel].notna()])
    timeseries = pd.Series(timeseries.values, index=index_range, name=channel)
    return timeseries

def get_closest_frame(timestamp, vid_timeseries, report_acc=None):
    """
    This function extracts a frame from a series so that it is as close as possible to a given timestamp
    :param timestamp: The time to match a frame to
    :param vid_timeseries: The time frames series to look at for a match
    :param report_acc: if set to 1, will report the accuracy of the match
    :return: index_of_lowest_diff , accuracy of match (if requested)
    """
    array = np.abs((vid_timeseries.to_numpy())-timestamp)
    index_of_lowest_diff = np.argmin(array)
    if report_acc == 1:
        accuracy = abs(vid_timeseries[index_of_lowest_diff] - timestamp)
        return index_of_lowest_diff, accuracy
    else:
        return index_of_lowest_diff

def synchronize_timestamps(arena_timestamps, ar_vidnames):
    '''
    :param arena_timestamps: a list of pathlib paths with arena timestamp files
    :param ar_vidnames: names of the arena videos as extracted in the code
    :return: arena_sync_df, a dataframe with the corresponding frames for each frame of the anchor video
    '''
    #read the timestamp files
    len_list = []
    df_list = []
    for p in arena_timestamps:
        df = pd.read_csv(p)
        df_list.append(df)
        len_list.append(len(df))

    #pick the longest as an anchor
    anchor_ind = len_list.index(max(len_list))
    anchor_vid = df_list[anchor_ind]
    anchor_vid_name = ar_vidnames[anchor_ind]
    #construct a synchronization dataframe
    arena_sync_df = pd.DataFrame(data=[],
                                 columns=ar_vidnames,
                                 index=range(len(anchor_vid)))

    #populate the df, starting with the anchor:
    arena_sync_df[arena_sync_df.columns[anchor_ind]] = range(len(anchor_vid))

    # now, the corresponding frames from the remaining videos:
    #start with removing the anchor video from the synchronization lists
    vids_to_sync = list(arena_sync_df.columns)
    del vids_to_sync[anchor_ind]
    anchor_df = df_list.pop(anchor_ind)
    df_to_sync = df_list
    #iterate over rows and videos to find the corresponding frames
    print('Synchronizing the different arena videos')
    for row in tqdm(arena_sync_df.index):
        anchor = anchor_vid.timestamp[row]
        for vid in range(len(df_to_sync)):
            frame_num = get_closest_frame(anchor,df_to_sync[vid])
            arena_sync_df.loc[row,vids_to_sync[vid]] = frame_num

    return arena_sync_df, anchor_vid_name

def arena_video_initial_thr(vid_path, threshold_value, show_frames=False):
    """
        This function works through an arena video to determine where the LEDs are on and when off
        :param threshold_value: value of the frame threshold
        :param show_frames: if true will show the video after thresholding
        :param  vid_path: Path to video. When ShowFrames is True a projection of the frames after threshold is presented

        :return: np.array with frame numbers and mean values after threshold
        """
    cap = cv2.VideoCapture(vid_path)
    all_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    flag = 0
    i = 0
    mean_values = []
    indexes = []
    while flag == 0:
        print('Frame number {} of {}'.format(i, all_frames), end='\r', flush=True)
        ret, frame = cap.read()
        if not ret:
            break
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        grey[grey < threshold_value] = 0
        mean_values.append(np.mean(grey))
        indexes.append(i)
        if show_frames:
            cv2.imshow('Thresholded_Frames', grey)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        i += 1
    cap.release()
    cv2.destroyAllWindows()
    frame_val = np.array((indexes, mean_values))
    return frame_val

def produce_frame_val_list(vid_paths,threshold_value):
    """
    :param vid_paths: a list of str paths to videos for analysis
    :param threshold_value: the threshold to use in order to concentrate on LEDs
    :return: frame_val_list: a list of mean pixel values for each frame after threshold
    """
    frame_val_list = []
    for vid in vid_paths:
        print(f'working on video {vid}')
        frame_val = arena_video_initial_thr(str(vid), threshold_value)
        frame_val_list.append(frame_val)
    print(f'done, frame_val_list contains {len(frame_val_list)} objects',flush=True)

    return frame_val_list

def four_video_concat(output_name, arena_vid1, arvid1_name, arena_vid2, arvid2_name,
                      eye_vid_left, eye_vid_right, arena_sync_df, start_frame, shortest_vid_length, format='H264'):
    """
    :param vid1: left down
    :param vid2: right down
    :param vid3: left up
    :param vid4: right up
    :param shortest_vid_length
    :param output_name: output file name
    :return: a concatenated video of the 4 inputs
    """
    cap0 = cv2.VideoCapture(arena_vid1)
    cap1 = cv2.VideoCapture(arena_vid2)
    cap2 = cv2.VideoCapture(eye_vid_left)
    cap3 = cv2.VideoCapture(eye_vid_right)
    anchor = start_frame
    last_ar1_f = arena_sync_df[arvid1_name][sync_vids.Arena_VideoFrame[start_frame]]
    last_ar2_f = arena_sync_df[arvid2_name][sync_vids.Arena_VideoFrame[start_frame]]
    last_le = sync_vids.Left_eye[start_frame]
    last_re = sync_vids.Right_eye[start_frame]
    fourcc = cv2.VideoWriter_fourcc(*format)
    out = cv2.VideoWriter((str(block_path) + r'\\' + output_name + '.mp4'),fourcc, 60.0, (640*2,480*2))
    try:
        while cap2.isOpened():
            ar1_f = arena_sync_df[arvid1_name][sync_vids.Arena_VideoFrame[anchor]]
            ar2_f = arena_sync_df[arvid2_name][sync_vids.Arena_VideoFrame[anchor]]
            l_eye_f = sync_vids.Left_eye[anchor]
            r_eye_f = sync_vids.Right_eye[anchor]

            if ar1_f != last_ar1_f + 1:
                cap0.set(1,ar1_f)
            ar1_ret, ar1_frame = cap0.read()
            ar1_frame = cv2.cvtColor(ar1_frame, cv2.COLOR_BGR2GRAY)
            ar1_frame = cv2.resize(ar1_frame,(640,480))
            last_ar1_f = ar1_f

            if ar2_f != last_ar2_f + 1:
                cap1.set(1,ar2_f)
            ar2_ret, ar2_frame = cap1.read()
            ar2_frame = cv2.cvtColor(ar2_frame, cv2.COLOR_BGR2GRAY)
            ar2_frame = cv2.resize(ar2_frame,(640,480))
            last_ar2_f = ar2_f

            if l_eye_f != last_le + 1:
                cap2.set(1,l_eye_f)
            le_ret, le_f = cap2.read()
            le_f = cv2.cvtColor(le_f, cv2.COLOR_BGR2GRAY)
            le_f = cv2.flip(le_f, 0)
            le_f = cv2.resize(le_f,(640,480))
            last_le = l_eye_f

            if r_eye_f != last_re + 1:
                cap3.set(1,r_eye_f)
            re_ret, re_f = cap3.read()
            re_f = cv2.cvtColor(re_f, cv2.COLOR_BGR2GRAY)
            re_f = cv2.flip(re_f, 0)
            re_f = cv2.resize(re_f,(640,480))
            last_re = r_eye_f

            eye_concat = np.hstack((le_f,re_f))
            ar_concat = np.hstack((ar1_frame, ar2_frame))
            vconcat = np.vstack((eye_concat, ar_concat))

            out.write(vconcat)
            anchor += 1
            print(f'writing video frame {anchor} out of {shortest_vid_length}', end='\r', flush=True)
            if anchor > shortest_vid_length-1:
                break
    except Exception:
        print(f'Encountered a problem with frame {anchor}, stopping concatenation')
    finally:
        cap0.release()
        cap1.release()
        cap2.release()
        cap3.release()
        out.release()
        cv2.destroyAllWindows()
        print('\n')
        print('Processed finished')

def eye_tracking_analysis(dlc_video_analysis_csv):
    """
    :param dlc_video_analysis_csv: the csv output of a dlc analysis of one video, already read by pandas with header=1
    :param bodyparts_list: a list of bodyparts as described in the dlc csv (i.e ['Pupil_12', 'Pupil_6'....])
    :returns ellipse_df: a DataFrame of ellipses parameters (center, width, height, phi, size) for each video frame

    """

    data = dlc_video_analysis_csv
    ellipses = []
    caudal_edge_ls= []
    rostral_edge_ls = []
    for row in range(1, len(data)-1):
        x_values = np.array(list([float(data['Pupil_12'][row]),
                                  float(data['Pupil_1'][row]),
                                  float(data['Pupil_3'][row]),
                                  float(data['Pupil_4'][row]),
                                  float(data['Pupil_6'][row]),
                                  float(data['Pupil_8'][row]),
                                  float(data['Pupil_10'][row])]))
        y_values = np.array(list([float(data['Pupil_12.1'][row]),
                                  float(data['Pupil_1.1'][row]),
                                  float(data['Pupil_3.1'][row]),
                                  float(data['Pupil_4.1'][row]),
                                  float(data['Pupil_6.1'][row]),
                                  float(data['Pupil_8.1'][row]),
                                  float(data['Pupil_10.1'][row])]))
        X = np.c_[x_values,y_values]

        el = LsqEllipse().fit(X)
        center, width, height, phi = el.as_parameters()
        center_x = center[0]
        center_y = center[1]
        ellipses.append([center_x,center_y,width,height,phi])
        caudal_edge = [
            float(data['Caudal_edge'][row]),
            float(data['Caudal_edge.1'][row])
        ]
        rostral_edge = [
            float(data['Rostral_edge'][row]),
            float(data['Rostral_edge.1'][row])
        ]
        caudal_edge_ls.append(caudal_edge)
        rostral_edge_ls.append(rostral_edge)
        if row % 50 == 0:
           print(f'just finished with {row} out of {len(data)-1}', end='\r',flush=True)
    ellipse_df = pd.DataFrame(columns = ['center_x','center_y', 'width', 'height', 'phi'], data = ellipses)
    a = np.array(ellipse_df['height'][:])
    b = np.array(ellipse_df['width'][:])
    ellipse_size_per_frame = a*b*math.pi
    ellipse_df['ellipse_size'] = ellipse_size_per_frame
    ellipse_df['rostral_edge'] = rostral_edge_ls
    ellipse_df['caudal_edge'] = caudal_edge_ls
    print('Done')
    return ellipse_df

def date_parser(epoch_ns):
    return pd.to_datetime(epoch_ns).tz_localize("UTC").tz_convert("Asia/Jerusalem")

def analyze_timestamp_csv(csv_path):
    tdf = pd.read_csv(
        csv_path,
        dtype={"timestamp": np.long},
        index_col="timestamp",
        parse_dates=True,
        date_parser=date_parser,
    )

    diff_secs = tdf.index.to_series().diff() / np.timedelta64(1, 's')
    freq = 1 / diff_secs
    fps = freq.mean()
    sd_fps = freq.std()
    n = len(tdf)

    return {"n": n, "diff": diff_secs, "freq": freq, "mean_fps": fps, "sd_fps": sd_fps, "df": tdf}

def four_arena_video_concat(output_name, arena_vid1, arvid1_name, arena_vid2, arvid2_name,
                            arena_vid3, arvid3_name, arena_vid4, arvid4_name, arena_sync_df,
                            start_frame, shortest_vid_length, format='H264'):
    """
    :param vid1: left down
    :param vid2: right down
    :param vid3: left up
    :param vid4: right up
    :param shortest_vid_length
    :param output_name: output file name
    :return: a concatenated video of the 4 inputs
    """
    cap0 = cv2.VideoCapture(arena_vid1)
    cap1 = cv2.VideoCapture(arena_vid2)
    cap2 = cv2.VideoCapture(arena_vid3)
    cap3 = cv2.VideoCapture(arena_vid4)
    anchor = start_frame
    last_ar1_f = arena_sync_df[arvid1_name][sync_vids.Arena_VideoFrame[start_frame]]
    last_ar2_f = arena_sync_df[arvid2_name][sync_vids.Arena_VideoFrame[start_frame]]
    last_ar3_f = arena_sync_df[arvid3_name][sync_vids.Arena_VideoFrame[start_frame]]
    last_ar4_f = arena_sync_df[arvid4_name][sync_vids.Arena_VideoFrame[start_frame]]
    fourcc = cv2.VideoWriter_fourcc(*format)
    out = cv2.VideoWriter((str(block_path) + r'\\' + output_name + '.mp4'),fourcc, 60.0, (640*2,480*2))
    try:
        while cap2.isOpened():
            ar1_f = arena_sync_df[arvid1_name][sync_vids.Arena_VideoFrame[anchor]]
            ar2_f = arena_sync_df[arvid2_name][sync_vids.Arena_VideoFrame[anchor]]
            ar3_f = arena_sync_df[arvid3_name][sync_vids.Arena_VideoFrame[anchor]]
            ar4_f = arena_sync_df[arvid4_name][sync_vids.Arena_VideoFrame[anchor]]

            if ar1_f != last_ar1_f + 1:
                cap0.set(1,ar1_f)
            ar1_ret, ar1_frame = cap0.read()
            ar1_frame = cv2.cvtColor(ar1_frame, cv2.COLOR_BGR2GRAY)
            ar1_frame = cv2.resize(ar1_frame,(640,480))
            last_ar1_f = ar1_f

            if ar2_f != last_ar2_f + 1:
                cap1.set(1,ar2_f)
            ar2_ret, ar2_frame = cap1.read()
            ar2_frame = cv2.cvtColor(ar2_frame, cv2.COLOR_BGR2GRAY)
            ar2_frame = cv2.resize(ar2_frame,(640,480))
            last_ar2_f = ar2_f

            if ar3_f != last_ar3_f + 1:
                cap1.set(1,ar3_f)
            ar3_ret, ar3_frame = cap2.read()
            ar3_frame = cv2.cvtColor(ar3_frame, cv2.COLOR_BGR2GRAY)
            ar3_frame = cv2.resize(ar3_frame,(640,480))
            last_ar3_f = ar3_f

            if ar4_f != last_ar4_f + 1:
                cap1.set(1,ar4_f)
            ar4_ret, ar4_frame = cap3.read()
            ar4_frame = cv2.cvtColor(ar4_frame, cv2.COLOR_BGR2GRAY)
            ar4_frame = cv2.resize(ar4_frame,(640,480))
            last_ar4_f = ar4_f


            eye_concat = np.hstack((ar3_frame,ar4_frame))
            ar_concat = np.hstack((ar1_frame, ar2_frame))
            vconcat = np.vstack((eye_concat, ar_concat))

            out.write(vconcat)
            anchor += 1
            print(f'writing video frame {anchor} out of {shortest_vid_length}', end='\r', flush=True)
            if anchor > shortest_vid_length-1:
                break
    except Exception:
        print(f'Encountered a problem with frame {anchor}, stopping concatenation')
    finally:
        cap0.release()
        cap1.release()
        cap2.release()
        cap3.release()
        out.release()
        cv2.destroyAllWindows()
        print('\n')
        print('Processed finished')

In [4]:
"""
This notebook should be the main sequence for incoming experimental data - there are prerequisites for a functional process:
- the process is designed to work block-by-block
- Data will be arranged into block folders under animal folders, where each block contains the next structure:

                                       /----> arena_videos  ->[config.yaml , info.yaml] videos -> [video files, output.log] timestamps -> [csv of timestamps]

Animal_x ->date(xx_xx_xxxx) -> block_x -----> eye_videos >> LE\RE -> video folder with name -> [video.h264 , video.mp4 , params.json , timestamps.csv]

                                       \----> oe_files >> date_time(xxxx_xx_xx_xx-xx-xx) --> [events.csv] internal open ephys structure from here (NWB format only!!!)
                                                                                             /////////////////////
                                                                                         TODO: internal parsing of this file
"""

'\nThis notebook should be the main sequence for incoming experimental data - there are prerequisites for a functional process:\n- the process is designed to work block-by-block\n- Data will be arranged into block folders under animal folders, where each block contains the next structure:\n\n                                       /----> arena_videos  ->[config.yaml , info.yaml] videos -> [video files, output.log] timestamps -> [csv of timestamps]\n\nAnimal_x ->date(xx_xx_xxxx) -> block_x -----> eye_videos >> LE\\RE -> video folder with name -> [video.h264 , video.mp4 , params.json , timestamps.csv]\n\n                                       \\----> oe_files >> date_time(xxxx_xx_xx_xx-xx-xx) --> [events.csv] internal open ephys structure from here (NWB format only!!!)\n                                                                                             /////////////////////\n                                                                                         TODO: internal pa

In [5]:
animal_number = '1000'
experiment_date = '26_05_2021'
block = '0'
block_path = Path(rf'D:\experiments\Animal_{animal_number}\{experiment_date}\block_{block}')
print(f'Block Path: {block_path}')

Block Path: D:\experiments\Animal_1000\26_05_2021\block_0


In [6]:
# find names with '-' and replace it with '_'
arena_path = block_path / 'arena_videos'
arena_files = [x for x in arena_path.iterdir()]
for i in arena_files:
    if '-' in i.name:
        newname = i.name.replace('-','_')
        newpath = i.parent / newname
        i.replace(newpath)
arena_files= [x for x in arena_path.iterdir()]
arena_videos = [x for x in arena_files if x.suffix == '.mp4']
arena_timestamps = [x for x in arena_files if x.suffix == '.csv']
#arena_timestamps = [x for x in arena_timestamps if 'block' in x.name]
ar_vidnames = [i.name for i in arena_videos]
print(f'Arena video Names:')
print(*ar_vidnames, sep='\n')

Arena video Names:
calib_long_back_20210526_132415.mp4
calib_long_left_20210526_132415.mp4
calib_long_right_20210526_132415.mp4
calib_long_top_20210526_132415.mp4


In [ ]:
vid_path = block_path / 'eye_videos'
print('converting videos...')
convert_h264_mp4(str(vid_path))
print('Validating videos...')
validate_no_framedrop(str(vid_path))

In [ ]:
print('stamping LE video')
stamp_diff_videos(str(vid_path) + r'\LE' , 'LE')

In [ ]:
le_video = glob.glob(str(block_path) + r'\eye_videos\LE\**\*.mp4')
re_video = glob.glob(str(block_path) + r'\eye_videos\RE\**\*.mp4')

In [7]:
arena_timestamps


[WindowsPath('D:/experiments/Animal_1000/26_05_2021/block_0/arena_videos/calib_long_back_20210526_132415.csv'),
 WindowsPath('D:/experiments/Animal_1000/26_05_2021/block_0/arena_videos/calib_long_left_20210526_132415.csv'),
 WindowsPath('D:/experiments/Animal_1000/26_05_2021/block_0/arena_videos/calib_long_right_20210526_132415.csv'),
 WindowsPath('D:/experiments/Animal_1000/26_05_2021/block_0/arena_videos/calib_long_top_20210526_132415.csv')]

In [8]:
arena_sync_df, arena_anchor_vid_name = synchronize_timestamps(arena_timestamps, ar_vidnames)

100%|██████████| 12451/12451 [00:11<00:00, 1047.06it/s]


Synchronizing the different arena videos


In [9]:
print(f'The anchor video used was "{arena_anchor_vid_name}"')
arena_sync_df

The anchor video used was "calib_long_back_20210526_132415.mp4"


,calib_long_back_20210526_132415.mp4,calib_long_left_20210526_132415.mp4,calib_long_right_20210526_132415.mp4,calib_long_top_20210526_132415.mp4
0,0,0,1,1
1,1,1,1,1
2,2,2,2,2
3,3,3,3,3
4,4,4,4,4
...,...,...,...,...
12446,12446,12446,12446,12446
12447,12447,12447,12447,12447
12448,12448,12448,12448,12448
12449,12449,12449,12449,12449


In [22]:
# Utilize the open-ephys files by sorting the event times by channel
channeldict = {
    5 : 'L_eye_TTL',
    6 : 'Arena_TTL',
    8 : 'R_eye_TTL'
}
exp_date_time = os.listdir(fr'{str(block_path)}\oe_files')[0]
oe_events , oe_off_events = oe_events_parser(str(block_path) + rf'\oe_files\{exp_date_time}\events.csv',
                                             channeldict,
                                             export_path=str(block_path) + rf'\oe_files\{exp_date_time}\parsed_events.csv')
ts_list = []

for chan in list(oe_events.columns):
    ts = get_frame_timeseries(oe_events, str(chan))
    ts_list.append(ts)

In [23]:
ts_len_list = list(map(len, ts_list))
shortest_ind = ts_len_list.index(min(ts_len_list))
shortest_ind

0

In [14]:
block_start_time = ts_list[shortest_ind].values[0]
block_end_time = ts_list[shortest_ind].values[-1]

In [15]:
print(f'Arena video start: {block_start_time} \n'
      f'Arena video end: {block_end_time} \n'
      f'Block length: {block_end_time - block_start_time} Seconds')

Arena video start: 101.97853088378906 
Arena video end: 335.6025390625 
Block length: 233.62400817871094 Seconds


In [25]:
#Arena
arena_ff = ts_list[1].values[ts_list[1] > block_start_time][0]
arena_first_index = ts_list[1][ts_list[1] == arena_ff].index
arena_lf = ts_list[1].values[ts_list[1]<block_end_time][-1]
arena_last_index = ts_list[1][ts_list[1] == arena_lf].index
arena_sync_s = pd.Series(ts_list[1][arena_first_index.asi8[0] : arena_last_index.asi8[0]])

#Left eye
le_ff = ts_list[0].values[ts_list[0] > block_start_time][0]
le_first_index = ts_list[0][ts_list[0] == le_ff].index
le_lf = ts_list[0].values[-1]
le_last_index = ts_list[0][ts_list[0] == le_lf].index
le_sync_s = ts_list[0]

#Right eye
re_ff = ts_list[2].values[ts_list[2] > block_start_time][0]
re_first_index = ts_list[2][ts_list[2] == re_ff].index
re_lf = ts_list[2].values[-1]
re_last_index = ts_list[2][ts_list[2] == re_lf].index
re_sync_s = ts_list[2]

sync_time_starts = max([arena_ff, le_ff, re_ff])
sync_time_ends = min([arena_lf, le_lf, re_lf])

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [17]:
#Define Anchor signal
anchor_signal = np.arange(sync_time_starts, sync_time_ends, 1/60)

#Create the DataFrame
sync_vids = pd.DataFrame(data=None,
                         index=range(len(anchor_signal)),
                         columns=['Left_eye','Arena','Arena_VideoFrame','Right_eye'])

accuracy_report = pd.DataFrame(data=None,
                               index=range(len(anchor_signal)),
                               columns=['Left_eye','Arena','Right_eye'])

# define dictionary for timestamp retrieval
ts_dict = {'Left_eye': ts_list[0],
           'Arena': ts_list[1],
           'Right_eye':ts_list[2]}

#Iterate over the length of the dataframe and fit
for frame in tqdm(range(len(anchor_signal))):
    anchor_time = anchor_signal[frame]
    for vid in ['Left_eye', 'Right_eye', 'Arena']:
        f,a = get_closest_frame(anchor_time, ts_dict[vid], report_acc=1)
        sync_vids[vid][frame] = f
        accuracy_report[vid][frame] = a

NameError: name 'sync_time_starts' is not defined

In [ ]:
#TODO: organize the manual synchronization

sync_vids.Arena_VideoFrame = sync_vids.Arena
sync_vids.Right_eye = sync_vids.Right_eye
sync_vids.Left_eye = sync_vids.Left_eye
sync_vids

In [ ]:
fig, axs = plt.subplots()
fig.set_facecolor('white')
_ = accuracy_report['Arena'].plot()

In [ ]:
arena_videos

In [ ]:
arena_frame_val_list = produce_frame_val_list(arena_videos,250)

In [ ]:
l_eye_frame_val_list = produce_frame_val_list(le_video, 30)
r_eye_frame_val_list = produce_frame_val_list(re_video,30)

In [ ]:
l_eye_values = stats.zscore(l_eye_frame_val_list[0][1])
r_eye_values = stats.zscore(r_eye_frame_val_list[0][1])


In [ ]:
arena_brightness_df = pd.DataFrame(index = anchor_signal)
for ind, vid in enumerate(ar_vidnames):
    vid_val_arr = stats.zscore(arena_frame_val_list[ind][1])
    sync_list = sync_vids.Arena_VideoFrame.astype(int)
    sync_list[sync_list >= len(vid_val_arr)] = len(vid_val_arr)-1
    arena_brightness_df.insert(loc=0,
                               column=str(vid),
                               value=vid_val_arr[sync_list])

In [ ]:
arena_brightness_df

In [ ]:
eye_brightness_df = pd.DataFrame(index=anchor_signal)
sync_left = sync_vids.Left_eye.astype(int)
eye_brightness_df.insert(loc=0,
                         column='left_eye',
                         value=l_eye_values[(sync_vids.Left_eye.values.astype(int)-1)[0:len(anchor_signal)]])
eye_brightness_df.insert(loc=0,
                         column='right_eye',
                         value=r_eye_values[(sync_vids.Right_eye.values.astype(int)-1)[0:len(anchor_signal)]])

In [ ]:
sec = 3600
x = 0
region = range(0+x*sec, 3600+x*sec)

fig = plt.figure(figsize=[20,7])
fig.set_facecolor('xkcd:white')
plt.plot(eye_brightness_df.iloc[region].index, eye_brightness_df.iloc[region].right_eye, label='right')
plt.plot(eye_brightness_df.iloc[region].index, eye_brightness_df.iloc[region].left_eye, label='left')
plt.plot(eye_brightness_df.iloc[region].index, arena_brightness_df.iloc[region][arena_anchor_vid_name], label='arena anchor')
plt.plot(eye_brightness_df.iloc[region].index, arena_brightness_df.iloc[region][arena_brightness_df.columns[1]], label='arena2')
plt.plot(eye_brightness_df.iloc[region].index, arena_brightness_df.iloc[region][arena_brightness_df.columns[2]], label='arena3')
plt.plot(eye_brightness_df.iloc[region].index, arena_brightness_df.iloc[region][arena_brightness_df.columns[3]], label='arena4')
plt.vlines(eye_brightness_df.iloc[region].index, -20, -8, linestyles={'dashed'})
_ = plt.legend()

In [ ]:
trialrun = 0
bokeh_fig = figure(title='Synchronization Brightness Z scores',
                   x_axis_label='Time',
                   y_axis_label='Brightness Z scores',
                   plot_width=1500,
                   plot_height=700
                   )
bokeh_fig.line(eye_brightness_df.index, eye_brightness_df.right_eye,
               legend_label='right eye',
               line_width=1.5,
               line_color='red')
bokeh_fig.line(eye_brightness_df.index, eye_brightness_df.left_eye,
               legend_label='left eye',
               line_width=1.5,
               line_color='blue')

bokeh_fig.line(eye_brightness_df.index, arena_brightness_df[arena_anchor_vid_name],
               legend_label='Arena',
               line_width=1.5,
               line_color='black')
bokeh_fig.line(eye_brightness_df.index, trialrun,
               legend_label='tryme',
               line_color='green',
               line_width=4
               )

slider = Slider(start=0, end=10, value=1, step=.1, title="Stuff")
slider.js_on_change("value", CustomJS(code="""
    console.log('slider: value=' + this.value, this.toString())
"""))
layout = bokeh.layouts.column(slider,bokeh_fig)
show(layout)

In [ ]:
four_arena_video_concat('Sync_trial_tal_v0',
                  str(arena_videos[0]),ar_vidnames[0],
                  str(arena_videos[1]), ar_vidnames[1],
                  str(arena_videos[2]),ar_vidnames[2],
                  str(arena_videos[3]),ar_vidnames[3], arena_sync_df,
                  2000,3000)

In [ ]:
#arena_sync_df['block0_left_20210419_143946.mp4'][0]
ar_vidnames

In [ ]:
arena_sync_df.to_csv('sanity_check_df.csv')

In [ ]:
pd.DataFrame